In [1]:
library(Boruta)
library(dplyr)
library(readr)
library(randomForest)
library(pROC)

Warning message:
"package 'Boruta' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


Warning message:
"package 'pROC' was built under R version 4.4.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




In [2]:
##Read CSV file
whole_data <- read.csv("C8_Heart_featuremeta.csv")

#Subset DataFrame to include only specific cohort, if needed. Remove this line and adjust code accordingly.
subset_data <- whole_data[whole_data$`ATTRIBUTE_Cohort` == 1, ]

#Prints subset_data to verify it worked.
dim(subset_data)

[1]  164 3414

In [3]:
#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_G1_G3 <- subset_data %>%
  filter(ATTRIBUTE_Group_number %in% c('Group1', 'Group3'))

subset_data_G1_G3

X_G1_G3 <- subset_data_G1_G3[, 14:3414]
y_G1_G3 <- as.factor(subset(subset_data, ATTRIBUTE_Group_number %in% c("Group1", "Group3"))$ATTRIBUTE_Group_number)
X_G3_G1 <- subset_data_G1_G3[, 14:3414]
y_G3_G1 <- as.factor(subset(subset_data, ATTRIBUTE_Group_number %in% c("Group3", "Group1"))$ATTRIBUTE_Group_number)

set.seed(123)          #Seed set for reproducibility

sampleid,ATTRIBUTE_TYPE_tissue,ATTRIBUTE_Type,Tube_Number,ATTRIBUTE_Cohort,Mouse.ID,ATTRIBUTE_Group_number,ATTRIBUTE_Euthanasia_date,ATTRIBUTE_DPI,ATTRIBUTE_organ,⋯,X749.5031_3.04_5615,X611.053_2.47_5616,X502.8139_2.39_5617,X550.8507_2.49_5618,X539.8441_2.48_5619,X581.7023_2.46_5620,X232.1907_2.41_5621,X724.4499_3.17_5622,X204.0863_6.12_5623,X760.5834_2.86_5624
<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Upper_right_1_434_905_1_44943_210_heart_P268_Y_A1,Sample_heart,Sample,434,1,905,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_435_906_1_44943_210_heart_P268_Y_A2,Sample_heart,Sample,435,1,906,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_436_907_1_44943_210_heart_P268_Y_A3,Sample_heart,Sample,436,1,907,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_437_908_1_44943_210_heart_P268_Y_A4,Sample_heart,Sample,437,1,908,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_441_793_3_44943_210_heart_P268_Y_A8,Sample_heart,Sample,441,1,793,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_442_794_3_44943_210_heart_P268_Y_A9,Sample_heart,Sample,442,1,794,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_443_795_3_44943_210_heart_P268_Y_A10,Sample_heart,Sample,443,1,795,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_444_796_3_44943_210_heart_P268_Y_A11,Sample_heart,Sample,444,1,796,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
middle_top_1_380_897_1_44936_203_heart_P266_B_A1,Sample_heart,Sample,380,1,897,Group1,44936,203,heart,⋯,0,0,0,0,0,0,0,0,0,0


In [4]:
n_runs <- 3           #Number of times to run RF
top_n <- 50            #Top N features to keep from each run
set.seed(123)          #Seed set for reproducibility
feature_counts <- list()
importance_list <- list()

In [5]:
#Run Boruta for G1vsG3
boruta_G1vsG3 <- Boruta(X_G1_G3, y_G1_G3, doTrace = 2, maxRuns = 100)

###Get confirmed + tentative features
##Extract features (confirmed + tentative), this is needed for standard RF.
selected_features <- getSelectedAttributes(boruta_G1vsG3, withTentative = TRUE)

#####Get importance scores (optional, we're not going off these values, but the final RF values).
#boruta_stats <- attStats(boruta_G1vsG3)
#boruta_filtered <- boruta_stats[confirmed_features, ]
#boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

#####Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#write.csv(boruta_filtered, "Boruta_G1_G3_Selected_Features.csv", row.names = TRUE)

After 19 iterations, +9.4 secs: 

 rejected 3385 attributes: X100.0759_0.73_4272, X1001.6537_2.97_1138, X1005.6845_3.07_5596, X1005.685_3.05_2120, X1013.6537_2.87_2185 and 3380 more;

 still have 16 attributes left.


After 23 iterations, +9.5 secs: 

 rejected 2 attributes: X100.076_2.55_128, X570.4566_3.15_385;

 still have 14 attributes left.


After 73 iterations, +10 secs: 

 rejected 1 attribute: X511.374_3.41_423;

 still have 13 attributes left.




In [6]:
##Now using the Boruta Trained data to go into Random Forest. G1vsG3

#Subset original data to only Boruta-selected features
X_boruta_G1_G3 <- X_G1_G3[, selected_features]

# Train Random Forest on Boruta-selected features
rf_model_boruta_trained <- randomForest(X_boruta_G1_G3, y_G1_G3, ntree = 2000, importance = TRUE)

# Extract Mean Decrease Accuracy (MDA) from importance
importance_vals <- importance(rf_model_boruta_trained, type = 1)[, 1]  # Only MDA

# Convert to dataframe
importance_df <- data.frame(
  feature = names(importance_vals),
  mean_importance = importance_vals
)

#Filter for features with MDA >= 1.5
filtered <- subset(importance_df, mean_importance >= 1.5)

# Print and export to CSV
print(filtered)
write.csv(filtered, "1.5_cutoff_Boruta_G1_G3.csv", row.names = FALSE)

                              feature mean_importance
X373.1826_3.18_56   X373.1826_3.18_56       14.657346
X268.1036_0.34_74   X268.1036_0.34_74        9.245402
X287.0174_2.41_138 X287.0174_2.41_138        7.545119
X256.1902_2.85_154 X256.1902_2.85_154       10.623313
X478.2168_3.21_173 X478.2168_3.21_173        8.435277
X438.2068_2.82_256 X438.2068_2.82_256        9.650618
X511.3737_3.12_263 X511.3737_3.12_263       11.796369
X307.1721_2.83_268 X307.1721_2.83_268       11.660754
X153.0405_0.36_315 X153.0405_0.36_315       13.991440
X526.4306_3.15_323 X526.4306_3.15_323       11.302115
X119.0853_2.86_358 X119.0853_2.86_358        8.073068
X465.3778_3.17_407 X465.3778_3.17_407        8.452722
X614.4827_3.14_608 X614.4827_3.14_608        9.539647


In [7]:
###Run Boruta for G3vsG1
set.seed(123)
boruta_G3vsG1 <- Boruta(X_G3_G1, y_G3_G1, doTrace = 1, maxRuns = 100)

##Get confirmed + tentative features
confirmed_features <- getSelectedAttributes(boruta_G3vsG1, withTentative = TRUE)

##Get importance scores
boruta_stats <- attStats(boruta_G3vsG1)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

#Write to CSV
write.csv(boruta_filtered, "Boruta_G3_G1_Selected_Features.csv", row.names = TRUE)